In [1]:
import pandas as pd
import geopandas as gpd
from h3 import h3
import keplergl
from shapely.geometry import Polygon
import os
from glob import glob

In [14]:
class HexBins():
    
    def coordsToH3HexBinID(self, row, zoomLevel):
        #zoomLevel  = self.getBinSize()
        return h3.geo_to_h3(row['LAT'], row['LON'], zoomLevel)
    
    def addGeometry(self, row):
        points = h3.h3_to_geo_boundary(row['H3Index'], True)
        return Polygon(points)
    
    def __init__(self,directory):
        currentDirectory = os.chdir(directory)
        dataFiles = glob('*.csv')
        dfList = []
        for file in dataFiles:
            df = pd.read_csv(file)
            columns = ['CITY_TOWN_NAME','CRASH_DATE_TEXT','CRASH_TIME','LAT','LON']
            df.drop(df.columns.difference(columns), 1, inplace=True)
            df['datetime'] = pd.to_datetime(df['CRASH_DATE_TEXT'] + ' ' + df['CRASH_TIME'])
            df.dropna(inplace=True)
            df.set_index('datetime', inplace=True)
            dfList.append(df)
        self.data = pd.concat(dfList)
        self.data['H3Index'] = self.data.apply(self.coordsToH3HexBinID args = [int(input('Enter Zoom Level: '))], axis=1)
        return
    
    '''
    count number of events in each hexbin.
    returns a dataframe
    '''
    def groupHexbinsCounts(self):    
        self.groupedDF = self.data.groupby(by='H3Index').size().reset_index()
        self.groupedDF.columns = ['HexID', 'Count']
        self.groupedDF['geometry'] = groupedDF.apply(self.addGeometry, axis=1)
        return groupedDF
    
    
        
       
    
#    def mapData(self):
#        keplerMap = keplergl.KeplerGl(height = 800)
#        keplerMap.add_data(data = self.geoDF, name = 'AccidentClusters')
#        keplerMap
        

In [18]:
test = HexBins(r"C:\Users\Kevin\Documents\AccidentData\test")

C:\Users\Kevin\anaconda3\envs\gdal\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (25,29,30,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [55]:
keplerMap = keplergl.KeplerGl(height = 800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [56]:
keplerMap.add_data(test.data, name='Accidents')

In [58]:
keplerMap

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ckr030m', 'type': …

In [21]:
df = test.data.copy()

In [61]:
geoDF = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.LAT, df.LON))

In [58]:
df['H3Index'] = df.apply(h3.geo_to_h3(df['LAT'], df['LON'], 10), axis=1)

TypeError: cannot convert the series to <class 'float'>

In [62]:
geoDF.head()

,CITY_TOWN_NAME,CRASH_DATE_TEXT,CRASH_TIME,LAT,LON,geometry
datetime,,,,,,
2011-01-08 12:55:00,REVERE,01/08/2011,12:55 PM,42.428736,-71.017399,POINT (42.42874 -71.01740)
2011-01-12 12:23:00,MANCHESTER,01/12/2011,12:23 PM,42.590394,-70.793323,POINT (42.59039 -70.79332)
2011-01-12 12:23:00,MANCHESTER,01/12/2011,12:23 PM,42.590394,-70.793323,POINT (42.59039 -70.79332)
2011-01-08 13:14:00,WEST SPRINGFIELD,01/08/2011,1:14 PM,42.135914,-72.618836,POINT (42.13591 -72.61884)
2011-01-07 15:02:00,SAVOY,01/07/2011,3:02 PM,42.634329,-72.973417,POINT (42.63433 -72.97342)


In [57]:
h3.geo_to_h3(34, 65, 8)

'88434e04c5fffff'

In [72]:
geoDF['hex'] = df.apply(h3.geo_to_h3(geoDF.geometry.x.values, geoDF.geometry.y.values, 10), axis=1)

TypeError: only size-1 arrays can be converted to Python scalars

In [67]:
arr = geoDF.geometry.y.values

In [71]:
arr.shape

(210197,)

In [54]:
help(h3.geo_to_h3)

Help on function geo_to_h3 in module h3.api._api_template:

geo_to_h3(lat, lng, resolution)
    Return the cell containing the (lat, lng) point
    for a given resolution.
    
    Returns
    -------
    H3Cell

